In [1]:
import nltk
from pycorenlp import StanfordCoreNLP
import re
import codecs
import requests

In [2]:
with codecs.open('../data/raw/battle-cry-army-of-the-pharaohs.txt', 'rb', encoding='utf-8', errors='ignore') as f:
    lyrics = re.sub('[^a-zA-Z0-9\n\[\]]', ' ', f.read()).lower()

In [3]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [7]:
lines = [l for l in lyrics.split('\n') if (len(l) > 0 and l[0] != '[')]

In [8]:
tokens = []
for line in lines:
    output = nlp.annotate(line, properties={
            'annotators': 'tokenize, ssplit', \
            'outputFormat': 'json' \
        })
    tokens.append([t['word'] for s in output['sentences'] for t in s['tokens']])

In [9]:
tokens

[['hahahaha'],
 ['yeah', 'muthafuckas', 'we', 're', 'back'],
 ['pharaoh', 'clique', 'muthafucka', 'yeah'],
 ['des', 'devious', 'king', 'syze'],
 ['apathy', 'celph', 'titled', 'kamachi'],
 ['vinnie', 'p', 'outerspace'],
 ['7l', 'es'],
 ['i',
  'put',
  'you',
  'up',
  'on',
  'the',
  'iv',
  'not',
  'the',
  'roman',
  'numeral',
  '4'],
 ['but', 'the', 'iv', 'that', 'leads', 'to', 'the', 'funeral', 'floor'],
 ['wax', 'gets', 'melted', 'breaks', 'bones', 'fractures', 'pelvis'],
 ['speeds',
  'through',
  'space',
  'and',
  'cracks',
  'glass',
  'astronaut',
  'helmets'],
 ['face',
  'it',
  'motherfucker',
  'i',
  'can',
  'pay',
  'to',
  'get',
  'rid',
  'of',
  'you'],
 ['i',
  've',
  'got',
  'more',
  'heads',
  'in',
  'the',
  'hood',
  'than',
  'pagan',
  'rituals'],
 ['a', 'new', 'tyrannical', 'force', 'for', 'you', 'to', 'fear'],
 ['known', 'to', 'kill', 'and', 'keep', 'human', 'ears', 'as', 'souvenirs'],
 ['a', 'shape', 'shifter', 'face', 'slitter', 'paper', 'getter'

In [57]:
arpabet = nltk.corpus.cmudict.dict()

base_url = 'http://www.speech.cs.cmu.edu/cgi-bin/tools/logios/lextool.pl'

file = {'wordfile': ('words.txt', "\n".join([str(t).upper() for line in tokens for t in line if t not in arpabet.keys()]))}

res = requests.post(base_url, files=file, allow_redirects=True)

dict_re = re.compile(r"(?<=DICT ).*?\.dict")

dict_url = dict_re.search(res.text).group(0)

res_dict = requests.get(dict_url)

In [60]:
custom_dict = {line.split('\t')[0].lower(): line.split('\t')[1].split(' ') for line in res_dict.text.split('\n') if len(line) > 1}

In [61]:
def get_phonemes(word):
    try:
        return arpabet[word][0]
    except:
        try:
            return custom_dict[word]
        except:
            print(word)
            return word

phonemes = [[get_phonemes(t) for t in line] for line in tokens]

In [62]:
phonemes

[[['HH', 'AH', 'HH', 'AE', 'HH', 'AH', 'HH', 'AH']],
 [['Y', 'AE1'],
  ['M', 'AH', 'TH', 'AH', 'F', 'AH', 'K', 'AH', 'Z'],
  ['W', 'IY1'],
  ['R', 'EY1'],
  ['B', 'AE1', 'K']],
 [['F', 'EH1', 'R', 'OW0'],
  ['K', 'L', 'IY1', 'K'],
  ['M', 'AH', 'TH', 'AH', 'F', 'AH', 'K', 'AH'],
  ['Y', 'AE1']],
 [['D', 'EH1'],
  ['D', 'IY1', 'V', 'IY0', 'AH0', 'S'],
  ['K', 'IH1', 'NG'],
  ['S', 'AY', 'Z']],
 [['AE1', 'P', 'AH0', 'TH', 'IY0'],
  ['S', 'EH', 'L', 'F'],
  ['T', 'AY1', 'T', 'AH0', 'L', 'D'],
  ['K', 'AE', 'M', 'AH', 'CH', 'IY']],
 [['V', 'IH1', 'N', 'IY0'],
  ['P', 'IY1'],
  ['AW', 'T', 'ER', 'S', 'P', 'EY', 'S']],
 [['S', 'EH', 'V', 'AH', 'N', 'EH', 'L'], ['EH1', 'S']],
 [['AY1'],
  ['P', 'UH1', 'T'],
  ['Y', 'UW1'],
  ['AH1', 'P'],
  ['AA1', 'N'],
  ['DH', 'AH0'],
  ['IH', 'V'],
  ['N', 'AA1', 'T'],
  ['DH', 'AH0'],
  ['R', 'OW1', 'M', 'AH0', 'N'],
  ['N', 'UW1', 'M', 'ER0', 'AH0', 'L'],
  ['F', 'OW', 'R']],
 [['B', 'AH1', 'T'],
  ['DH', 'AH0'],
  ['IH', 'V'],
  ['DH', 'AE1', 'T'],
  [

In [13]:
import numpy as np
import pandas as pd

In [31]:
song_name = 'battle_cry'
pd.DataFrame({song_name: pd.Series([p for line in phonemes for word in line for p in word]).value_counts()})#.transpose()

,battle_cry
T,248
AH0,245
N,243
R,216
S,207
M,171
D,151
L,144
K,143
IH1,128
